# Finding a disease mutation

In this tutorial, we will identify a disease mutation from sequencing data of Dr James Lupski.

    
<img src="https://signature.bcm.edu/images/uploaded/full/1449086752427.jpeg" width=320>    

<center>https://en.wikipedia.org/wiki/James_R._Lupski</center>

Overview of workflow:

<img src="https://bchdb.nus.edu.sg/media/notebook/workflow.png">

For this tutorial, we will use a smaller reference genome (chromosome 5) for quicker processing, and a small subset of the input DNA sequences from Dr Lupski.

Let's take a look at the contents of the directory

In [1]:
ls -lh

total 178M
-rw-r--r-- 1 root root  17K Jul 23 14:44 '01 - Preparations for Finding a Disease Mutation.ipynb'
-rw-r--r-- 1 root root  17K Jul 23 14:44 '02 - Aligning the FASTQ File.ipynb'
-rw-r--r-- 1 root root  63K Jul 23 14:44 '03 - Variant Calling.ipynb'
-rw-r--r-- 1 root root  50K Jul 23 14:44 '04 - Annotation of Variants.ipynb'
-rw-r--r-- 1 root root 177M Jul 23 14:45  chr5.fa
-rw-r--r-- 1 root root 820K Jul 23 14:44  input.fq


Let us look at the 2 files that will be using

- chr5.fa - the human reference genome (chromosome 5)
- input.fq - the query sequences

Note: We are using a trimmed down Illumina exome dataset of Dr. James Lupski (SRR866988.sra) which has a disease causing mutation on chromosome 5

### Taking a peek at the reference fasta file

In [2]:
head chr5.fa

>chr5
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN


The fasta format is quite simple. The first line is the identifier which starts with '>'

The subsequent lines are DNA sequences. Here we see 'N's which means that the sequences are unknown.

OPTIONAL: We can also take a look at 10 lines of DNA sequence starting from the 100,000th line in the reference file. Here we use the tail command to list the lines starting from line number 100,0000 then pass it to the head command to show only 10 lines of the output from the tail command.

In [3]:
tail -n+100000 chr5.fa | head -n10 

gcgctgtttggttttctgttccttcgttagtttgctgaggatgatggctt
ccagcttcgttgatgtccctgcaaaatacatgatcttattcctttttatg
gctgcatggtattccatggtatatatgtaccacatagaaaatgggatttt
cttttctatcacattgtcaggctgcaaattttctgaacttttatgctcag
tttttccttttaaaactgaatgcctttaacagcatccacatcacatcttg
aatgctttgctgcttacaaattttttccaccagataccctaaatcatctc
tctgaagttcaaagtttcacagatctctagggcaggggcaaaatgcaacc
agtccccactaaaacataacaagagtcacctttgttcctgttcccaataa
gttcctcatcttcatctgagaccacctcagtctggatttcattgtccata
tcatcagcattttggtcaaagccattcaacaagtctctaagggtttccaa
tail: error writing 'standard output': Broken pipe


### Taking a look at the query sequence file (first 4 lines)

In [4]:
head -n4 input.fq

@SRRQ866988.19885082
CCAAGTAAGATTGAGCTTGAAGGCTGTTCTCATTTTGTAAAAACATAAGCTCAGGAAGTGTTGAAGATATTTTAACTCTACACTGAGACTT
+SRRQ866988.19885082
GIIGIIIIIIIIHIIIIIIIIIIIIIIIIIIGIIIIIIIIIIHIIIIIGIIIIEHBGGEGIIHIHIIIFIIIIHIIBHIIGEHIE<EII<G


### One sequence in a fastq file consists of 4 lines. 

- Line 1 - sequence identifier (starts with @)
- Line 2 - DNA sequence
- Line 3 - sequence identifier (starts with +)
- Line 4 - corresponding quality score (Phred score 0-93 + 33)

For the quality score, the following characters encode the lowest to highest scores

<pre> !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~ </pre>

For more information, see https://en.wikipedia.org/wiki/FASTQ_format


## Checking the quality of the FASTQ sequences

It is a good practice to check the quality of the sequences by plotting the quality (Q) scores by the position. In general, a Q score of > 30 is good.

To generate a plot, we will use the FASTX Toolkit (http://hannonlab.cshl.edu/fastx_toolkit/index.html).

In [5]:
fastqc input.fq

Started analysis of input.fq
Approx 25% complete for input.fq
Approx 50% complete for input.fq
Approx 80% complete for input.fq
Analysis complete for input.fq


In [6]:
ls -lh

total 178M
-rw-r--r-- 1 root root  17K Jul 23 14:44 '01 - Preparations for Finding a Disease Mutation.ipynb'
-rw-r--r-- 1 root root  17K Jul 23 14:44 '02 - Aligning the FASTQ File.ipynb'
-rw-r--r-- 1 root root  63K Jul 23 14:44 '03 - Variant Calling.ipynb'
-rw-r--r-- 1 root root  50K Jul 23 14:44 '04 - Annotation of Variants.ipynb'
-rw-r--r-- 1 root root 177M Jul 23 14:45  chr5.fa
-rw-r--r-- 1 root root 820K Jul 23 14:44  input.fq
-rw-r--r-- 1 root root 225K Jul 23 14:46  input_fastqc.html
-rw-r--r-- 1 root root 235K Jul 23 14:46  input_fastqc.zip


We have now generated a plot (input.png) that we can view

# Building the index for alignment

Before we can align the query sequence, we need to build the index for alignment. In this case, we will be using the chr5.fa file

One of the most popular programs for alignment is BWA, written by Heng Li (now at the Broad Institute, MIT). This alignment program makes use of an algorithm called Burrows-Wheeler transform to speed up the alignment process, allowing millions of sequences to be aligned to a reference genome in a reasonable amount of time.

To access the program in a shared environment, we will load the BWA module and run BWA to see what the options are when running this program

In [7]:
bwa


Program: bwa (alignment via Burrows-Wheeler transformation)
Version: 0.7.17-r1188
Contact: Heng Li <lh3@sanger.ac.uk>

Usage:   bwa <command> [options]

Command: index         index sequences in the FASTA format
         mem           BWA-MEM algorithm
         fastmap       identify super-maximal exact matches
         pemerge       merge overlapping paired ends (EXPERIMENTAL)
         aln           gapped/ungapped alignment
         samse         generate alignment (single ended)
         sampe         generate alignment (paired ended)
         bwasw         BWA-SW for long queries

         shm           manage indices in shared memory
         fa2pac        convert FASTA to PAC format
         pac2bwt       generate BWT from PAC
         pac2bwtgen    alternative algorithm for generating BWT
         bwtupdate     update .bwt to the new format
         bwt2sa        generate SA from BWT and Occ

Note: To use BWA, you need to first index the genome with `bwa index'.
      There are

: 1

Prior to the alignment, the reference genome must be indexed. This process may take several hours if indexing the full human genome (~4 GB), so we will use this smaller file to speed things up. In this case, we index only chromosome 5, as the disease mutation is located on this chromosome

In [8]:
bwa index chr5.fa

[bwa_index] Pack FASTA... 1.03 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=363076518, availableWord=37547260
[BWTIncConstructFromPacked] 10 iterations done. 61936310 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 114423334 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 161069910 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 202525462 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 239367270 characters processed.
[BWTIncConstructFromPacked] 60 iterations done. 272108326 characters processed.
[BWTIncConstructFromPacked] 70 iterations done. 301204614 characters processed.
[BWTIncConstructFromPacked] 80 iterations done. 327061462 characters processed.
[BWTIncConstructFromPacked] 90 iterations done. 350039062 characters processed.
[bwt_gen] Finished constructing BWT in 97 iterations.
[bwa_index] 91.37 seconds elapse.
[bwa_index] Update BWT... 0.97 sec
[bwa_index

The indexing process generates several files (.amb, .ann, .bwt, .pac, .sa), prefixed by the name of the input reference file (in this case, chr5.disease.fasta)

In [9]:
ls -lh

total 481M
-rw-r--r-- 1 root root  15K Jul 23 14:48 '01 - Preparations for Finding a Disease Mutation.ipynb'
-rw-r--r-- 1 root root  17K Jul 23 14:44 '02 - Aligning the FASTQ File.ipynb'
-rw-r--r-- 1 root root  63K Jul 23 14:44 '03 - Variant Calling.ipynb'
-rw-r--r-- 1 root root  50K Jul 23 14:44 '04 - Annotation of Variants.ipynb'
-rw-r--r-- 1 root root 177M Jul 23 14:45  chr5.fa
-rw-r--r-- 1 root root  588 Jul 23 14:48  chr5.fa.amb
-rw-r--r-- 1 root root   44 Jul 23 14:48  chr5.fa.ann
-rw-r--r-- 1 root root 174M Jul 23 14:48  chr5.fa.bwt
-rw-r--r-- 1 root root  44M Jul 23 14:48  chr5.fa.pac
-rw-r--r-- 1 root root  87M Jul 23 14:49  chr5.fa.sa
-rw-r--r-- 1 root root 820K Jul 23 14:44  input.fq
-rw-r--r-- 1 root root 225K Jul 23 14:46  input_fastqc.html
-rw-r--r-- 1 root root 235K Jul 23 14:46  input_fastqc.zip
